In [2]:
import pandas as pd

In [6]:
rf_df = pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
rf_df

FileNotFoundError: [Errno 2] No such file or directory: 'F-F_Research_Data_Factors_daily.CSV'

In [3]:
df = pd.read_csv('Gemini_BTCUSD_d.csv', skiprows=1)
df = df.iloc[:, 1:]
df = df.sort_values('date', ascending=True)
df

,date,symbol,open,high,low,close,Volume BTC,Volume USD
3686,2015-10-08 04:00:00,BTC/USD,0.00,245.00,0.00,243.60,34.754703,8.466246e+03
3685,2015-10-09 04:00:00,BTC/USD,243.60,249.97,243.60,245.51,61.587068,1.512024e+04
3684,2015-10-10 04:00:00,BTC/USD,245.51,246.30,244.60,246.30,30.870549,7.603416e+03
3683,2015-10-11 04:00:00,BTC/USD,246.30,249.50,245.96,248.98,22.747091,5.663571e+03
3682,2015-10-12 04:00:00,BTC/USD,248.98,248.98,245.75,245.75,71.047743,1.745998e+04
...,...,...,...,...,...,...,...,...
4,2024-11-18 00:00:00,BTC/USD,89824.74,92667.79,89372.20,90520.03,1932.319076,1.749136e+08
3,2024-11-19 00:00:00,BTC/USD,90520.03,94211.99,90400.00,92431.50,1509.898605,1.395622e+08
2,2024-11-20 00:00:00,BTC/USD,92431.50,95000.00,91613.30,94376.05,1667.173738,1.573413e+08
1,2024-11-21 00:00:00,BTC/USD,94376.05,99000.00,94155.83,98447.95,2088.066242,2.055658e+08


In [4]:
# create the signal for the trading range breakout
n = 50 # tracking back for 50 days

df['SUP'] = df['close'].shift(1).rolling(window=50).min()
df['RES'] = df['close'].shift(1).rolling(window=50).max()
df

,date,symbol,open,high,low,close,Volume BTC,Volume USD,SUP,RES
3686,2015-10-08 04:00:00,BTC/USD,0.00,245.00,0.00,243.60,34.754703,8.466246e+03,NaN,NaN
3685,2015-10-09 04:00:00,BTC/USD,243.60,249.97,243.60,245.51,61.587068,1.512024e+04,NaN,NaN
3684,2015-10-10 04:00:00,BTC/USD,245.51,246.30,244.60,246.30,30.870549,7.603416e+03,NaN,NaN
3683,2015-10-11 04:00:00,BTC/USD,246.30,249.50,245.96,248.98,22.747091,5.663571e+03,NaN,NaN
3682,2015-10-12 04:00:00,BTC/USD,248.98,248.98,245.75,245.75,71.047743,1.745998e+04,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4,2024-11-18 00:00:00,BTC/USD,89824.74,92667.79,89372.20,90520.03,1932.319076,1.749136e+08,60280.67,91080.23
3,2024-11-19 00:00:00,BTC/USD,90520.03,94211.99,90400.00,92431.50,1509.898605,1.395622e+08,60280.67,91080.23
2,2024-11-20 00:00:00,BTC/USD,92431.50,95000.00,91613.30,94376.05,1667.173738,1.573413e+08,60280.67,92431.50
1,2024-11-21 00:00:00,BTC/USD,94376.05,99000.00,94155.83,98447.95,2088.066242,2.055658e+08,60280.67,94376.05


In [9]:
df = df.dropna()

In [13]:
df['Act'] = df.apply(lambda row: -1*row['close'] if row['close'] < row['SUP'] else (row['close'] if row['close'] > row['RES'] else 0), axis=1)
df

,date,symbol,open,high,low,close,Volume BTC,Volume USD,SUP,RES,Act
3636,2015-11-27 04:00:00,BTC/USD,350.82,369.04,348.79,353.78,1084.180175,3.835613e+05,243.60,441.95,0.00
3635,2015-11-28 04:00:00,BTC/USD,353.78,359.99,352.53,357.01,101.355031,3.618476e+04,245.51,441.95,0.00
3634,2015-11-29 04:00:00,BTC/USD,357.01,380.00,354.01,373.87,683.993384,2.557246e+05,245.75,441.95,0.00
3633,2015-11-30 04:00:00,BTC/USD,373.87,380.99,367.53,377.32,1346.303446,5.079872e+05,245.75,441.95,0.00
3632,2015-12-01 04:00:00,BTC/USD,377.32,379.99,349.00,352.52,545.953655,1.924596e+05,245.75,441.95,0.00
...,...,...,...,...,...,...,...,...,...,...,...
4,2024-11-18 00:00:00,BTC/USD,89824.74,92667.79,89372.20,90520.03,1932.319076,1.749136e+08,60280.67,91080.23,0.00
3,2024-11-19 00:00:00,BTC/USD,90520.03,94211.99,90400.00,92431.50,1509.898605,1.395622e+08,60280.67,91080.23,92431.50
2,2024-11-20 00:00:00,BTC/USD,92431.50,95000.00,91613.30,94376.05,1667.173738,1.573413e+08,60280.67,92431.50,94376.05
1,2024-11-21 00:00:00,BTC/USD,94376.05,99000.00,94155.83,98447.95,2088.066242,2.055658e+08,60280.67,94376.05,98447.95
